<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#特征检测" data-toc-modified-id="特征检测-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>特征检测</a></span><ul class="toc-item"><li><span><a href="#Harris-角点检测" data-toc-modified-id="Harris-角点检测-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Harris 角点检测</a></span><ul class="toc-item"><li><span><a href="#亚像素级精确度的角点" data-toc-modified-id="亚像素级精确度的角点-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>亚像素级精确度的角点</a></span></li></ul></li><li><span><a href="#Shi-Tomasi-角点检测" data-toc-modified-id="Shi-Tomasi-角点检测-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Shi-Tomasi 角点检测</a></span></li></ul></li><li><span><a href="#尺度空间极值检测" data-toc-modified-id="尺度空间极值检测-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>尺度空间极值检测</a></span><ul class="toc-item"><li><span><a href="#背景" data-toc-modified-id="背景-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>背景</a></span></li><li><span><a href="#原理" data-toc-modified-id="原理-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>原理</a></span></li></ul></li></ul></div>

## 特征检测
> 粗略地讲，特征就是有意义的图像区域，该区域具有独特性或易于识别性。因此，角点及高密度区域是很好的特征，而大量重复的模式或低密度区域则不是好的特征。  
> 例如边缘可以将图像分成两个区域，可以看作好的特征；斑点(与周围有很大差别的图像区域)也是有意义的特征  

>> OpenCVZ中常见的特征检测和提取算法：
* Harris: 该算法用于检测角点
* SIFT: 该算法用于检测斑点(blob)
* SURF: 该算法用于检测斑点
* FAST: 该算法用于检测角点
* BRIEF: 该算法用于检测斑点

### Harris 角点检测
> 角点的一个特性：向任何方向移动变化都很大  
> Harris 角点检测的结果是一个由角点分数构成的灰度图像。选取适当的阈值对结果图像进行二值化就检测到了图像中的角点。

> 函数原型：
```python
cornerHarris(src, blockSize, ksize, k[, dst[, borderType]]) -> dst
```

> 参数说明:
```python
* src： 单通道8位或浮点型源图片
* blockSize： 角点检测中要考虑的领域大小(int)
* ksize： Sobel求导中使用的窗口大小(int)；该参数定义了角点检测的敏感度，其取值必须是介于3和31之间的奇数
* k： Harris角点检测方程中的自由参数，取值参数为 [0.04，0.06](double)
* borderType： 边框类型，可选参数
```

> 返回值说明:
```python
* dst: 一个浮点型图像，大小跟输入相同(灰度图像坐标与原图像对应)，表示Harris角点检测的结果，浮点值越高，表示越可能是角点
```

> 示例:
```python
import cv2
import numpy as np
img = cv2.imread('chessboard.png')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # 1. Harris角点检测基于灰度图像
dst = cv2.cornerHarris(gray, 2, 3, 0.04)      # 2. Harris角点检测
img[dst > 0.01 * dst.max()] = [0, 0, 255]     # 3. 角点标记为红色（阈值可动态调整）
cv2.imshow('dst', img)
cv2.waitKey()
```

> 小结:
* Harris角点检测是一种基于灰度图像的角点检测方法
* Harris具有旋转不变特性，即使图片发生了旋转，也能找到同样的角点
* 其原理是先选取一个矩形块，移动这个矩形块，然后计算像素间的差值

#### 亚像素级精确度的角点
> OpenCV 提供了函数 cv2.cornerSubPix() 用于亚像素级别的角点检测  
> 首先要找到Harris角点，然后将角点的重心传给这个函数进行修正; 在使用这个函数时要定义一个迭代停止条件。当迭代次数达到或者精度条件满足后迭代就会停止

> 函数原型：
```python
cornerSubPix(image, corners, winSize, zeroZone, criteria) -> corners
```




### Shi-Tomasi 角点检测
> Shi-Tomasi算法是对Harris角点检测的改进；在图像中寻找具有大特征值的角点
> Shi-Tomasi算法的一个好处是，它可以直接得到角点的坐标值corners

> 函数原型：
```python
goodFeaturesToTrack(image, maxCorners, qualityLevel, minDistance[, corners[, mask[, blockSize[, useHarrisDetector[, k]]]]]) -> corners
```

> 函数说明:
```
* 算法首先进行角点检测，类似Harris算法，返回一个角点检测的浮点型响应结果，浮点值越高，越有可能是角点  
* 其次需要指定最多找多少个角点，也就是第2个参数`maxCorners`
* 然后输入阈值参数`qualityLevel`，范围在0~1之间，这个值跟Harris算法中的阈值0.01×dst.max()含义相同，可以理解成角点的概率，只有大于这个阈值的点才会被认为是角点  
* 最后再指定角点间的最小距离`minDistance`  
```

> 参数说明:
```python
* maxCorners :  最大数目的角点数  
* qualityLevel： 该参数指出最低可接受的角点质量，是一个百分数。具体来说，如果最好的角点质量=1500，而qualityLevel = 0.01，那么角点质量<15的就都会被拒掉
* minDistance： 角点之间最小的欧拉距离，对于初选出的角点而言，如果在其周围minDistance范围内存在其他更强角点，则将此角点删除    
* corners： 检测到的角点的输出向量  
* mask： 可选参数，给出ROI。该参数与原图尺寸相同且类型为CV_8UC1，指示出需要进行特征检测的区域，如在人脸检测中检测到的人脸区域进行特征点提取
* blocksize：在每一个像素的领域中计算derivative covariation matrix的平均块的大小。即计算M用到的块大小  
```

> 示例:
```python
import cv2
import numpy as np
img = cv2.imread("../img/blox.jpg")
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
corners = cv2.goodFeaturesToTrack(gray, 20, 0.1, 10)   # 试图寻找20个角点(返回结果<=20)，并且角点之间的距离不小于10个像素； 返回值 corners 是一个三维ndarray [20, 1, 3]
corners = np.int0(corners)
for i in corners:
    x, y = i.ravel()     # 扁平化操作，压缩到一维
    cv2.circle(img, (x, y), 4, (0, 0, 255), -1)
cv2.imshow('dst', img)
cv2.waitKey(0)
cv2.destroyAllWindows()
```

## 尺度空间极值检测

### 背景
> Harris算法具有旋转不变性，即使图片发生了旋转，也能找到同样的角点；但是，如果对图像进行缩放后，角点可能就不再是角点了；例如，在一副小图中使用一个小的窗口可以检测到一个角点，但是如果图像被放大，再使用同样的窗口就检测不到角点了  

### 原理
> 不同的尺度空间不能使用相同的窗口检测极点值；对小的角点要用小的窗口，对大的角点只能使用大的窗口，为了达到这个目的，需要使用尺度空间滤波器（尺度空间滤波器可以使用一些具有不同方差的高斯卷积核构成）  
> 使用具有不同方差值的高斯拉普拉斯算子(LoG)对图像进行卷积，(LoG)由于具有不同的方差值所以可以用来检测不同大小的斑点（当(LoG)的方差与斑点直径相等时能够使斑点完全平滑）。简单来说方差就是一个尺度变换因子。  
> 高斯方差的大小与窗口的大小存在一个倍数关系：窗口大小等于 6 倍方差加 1，所以方差的大小也决定了窗口大小；使用一个小方差的高斯卷积核是可以很好的检测出小的角点，而使用大方差的高斯卷积核时可以很好的检测除大的角点 
